In [6]:
# straightforward linear regression with 4 outputs with and without sorting of outputs

In [7]:
# Experiment 2 - 1 output layer with 1 loss function - mse. and do hyper parameter tuning.
from utils.data_preprocess import load_data, load_single_leakage_model_data
from utils.module import model_eval, hyper_model, model_comparison, linear_regression, numpy_to_tensor, benchmark_linear_model
import itertools
import pandas as pd 
import yaml
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Dense, Input
import keras_tuner as kt
from tensorflow import keras
from keras import layers
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [8]:

single_leakage, two_leakage = load_data()
# two_leakage["leak_1"] = 1
# two_leakage["leak_2"] = 1

# single_leakage["leak_1"] = 1
# single_leakage["leak_2"] = 0

data = pd.concat([single_leakage, two_leakage], axis=0)
data['x2'] = data['x2'].replace(np.nan, 8024)
data['y2'] = data['y2'].replace(np.nan, 2616.5)

data = data.drop(columns=['mfc6_residual',
       'mfc7_residual', 'mfc8_residual', 'mfc9_residual', 'mfc10_residual',
       'mfc1_residual', 'mfc2_residual', 'mfc3_residual', 'mfc4_residual',
       'mfc5_residual', 'total flow rate'])

print(data.columns)
print(data.isna().sum())

y = data[['x1', 'y1', 'x2', 'y2']]
x = data.drop(['x1', 'y1', 'x2', 'y2'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1) 

y1_train = y_train[['x1', 'y1', 'x2', 'y2']]
# y2_train = y_train[['leak_1', 'leak_2']]
y1_test = y_test[['x1', 'y1', 'x2', 'y2']]
# y2_test = y_test[['leak_1', 'leak_2']]
y1_val = y_val[['x1', 'y1', 'x2', 'y2']]
# y2_val = y_val[['leak_1', 'leak_2']]

def coords_swap(y1):
    s = y1['x2'] < y1['x1']
    y1.loc[s, ['x1','x2']] = y1.loc[s, ['x2','x1']].values
    y1.loc[s, ['y1','y2']] = y1.loc[s, ['y2','y1']].values
    return y1

# y1_data = [y1_train, y1_val, y1_test]
# y1_data_types = ['y1_train', 'y1_val', 'y1_test']
# for y1_data_types, y1 in zip(y1_data_types, y1_data):
#     y1_data_types = coords_swap(y1)

y1_columns = y1_train.columns
# y2_columns = y2_train.columns
X_columns = X_train.columns

scaler_coords1 = StandardScaler()
y1_train = scaler_coords1.fit_transform(y1_train)
y1_test = scaler_coords1.transform(y1_test)
y1_val = scaler_coords1.transform(y1_val)

y_train_sc = pd.DataFrame(y1_train, columns=y1_columns)
y_test_sc = pd.DataFrame(y1_test, columns=y1_columns)
y_val_sc = pd.DataFrame(y1_val, columns=y1_columns)

# y1_train['x2'] = y1_train['x2'].replace(np.nan, -5)
# y1_train['y2'] = y1_train['y2'].replace(np.nan, -5)

# y1_test['x2'] = y1_test['x2'].replace(np.nan, -5)
# y1_test['y2'] = y1_test['y2'].replace(np.nan, -5)

# y1_val['x2'] = y1_val['x2'].replace(np.nan, -5)
# y1_val['y2'] = y1_val['y2'].replace(np.nan, -5)
# Not sure if 0 is good enough or try generating a random number

# scaler_coords2 = StandardScaler()
# y2_train = scaler_coords2.fit_transform(y2_train)
# y2_test = scaler_coords2.fit_transform(y2_test)
# y2_val = scaler_coords2.transform(y2_val)

# y2_train = pd.DataFrame(y2_train, columns=y2_columns)
# y2_test = pd.DataFrame(y2_test, columns=y2_columns)
# y2_val = pd.DataFrame(y2_val, columns=y2_columns)

# y2_train = y2_train.reset_index().drop(columns='sample_number')
# y2_val = y2_val.reset_index().drop(columns='sample_number')
# y2_test = y2_test.reset_index().drop(columns='sample_number')

# y_train_sc = pd.concat([y1_train, y2_train], axis=1)
# y_test_sc = pd.concat([y1_test, y2_test], axis=1)
# y_val_sc = pd.concat([y1_val, y2_val], axis=1)

scaler_flows = StandardScaler()
X_train = scaler_flows.fit_transform(X_train)
X_test = scaler_flows.transform(X_test)
X_val = scaler_flows.transform(X_val)

Index(['x1', 'y1', 'MFC6', 'MFC7', 'MFC8', 'MFC9', 'MFC10', 'MFC1', 'MFC2',
       'MFC3', 'MFC4', 'MFC5', 'x2', 'y2'],
      dtype='object')
x1       0
y1       0
MFC6     0
MFC7     0
MFC8     0
MFC9     0
MFC10    0
MFC1     0
MFC2     0
MFC3     0
MFC4     0
MFC5     0
x2       0
y2       0
dtype: int64


In [9]:
# Fitting a simple linear regression model
reg = LinearRegression().fit(X_train, y_train_sc)


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
y_predictions_train = reg.predict(X_train)
# print("train", "{:10.4f}".format(mean_squared_error(y_train, y_predictions, squared=True)))
y_predictions_val = reg.predict(X_val)
# print("val", "{:10.4f}".format(mean_squared_error(y_val, y_predictions, squared=True)))
y_predictions = reg.predict(X_test)

y_predictions_train[:,-2][np.abs(y_predictions_train[:,-2]) < 0.5] = 0
y_predictions_train[:,-2][np.abs(y_predictions_train[:,-2]) > 0.5] = 1
y_predictions_train[:,-1][np.abs(y_predictions_train[:,-1]) < 0.5] = 0
y_predictions_train[:,-1][np.abs(y_predictions_train[:,-1]) > 0.5] = 1

y_predictions_val[:,-2][np.abs(y_predictions_val[:,-2]) < 0.5] = 0
y_predictions_val[:,-2][np.abs(y_predictions_val[:,-2]) > 0.5] = 1
y_predictions_val[:,-1][np.abs(y_predictions_val[:,-1]) < 0.5] = 0
y_predictions_val[:,-1][np.abs(y_predictions_val[:,-1]) > 0.5] = 1

y_predictions[:,-2][np.abs(y_predictions[:,-2]) < 0.5] = 0
y_predictions[:,-2][np.abs(y_predictions[:,-2]) > 0.5] = 1
y_predictions[:,-1][np.abs(y_predictions[:,-1]) < 0.5] = 0
y_predictions[:,-1][np.abs(y_predictions[:,-1]) > 0.5] = 1

loss_test = "{:10.4f}".format(mean_squared_error(y_test_sc, y_predictions, squared=True))
metric_test = "{:10.4f}".format(mean_absolute_error(y_test_sc, y_predictions))

loss_val = "{:10.4f}".format(mean_squared_error(y_val_sc, y_predictions_val, squared=True))
metric_val = "{:10.4f}".format(mean_absolute_error(y_val_sc, y_predictions_val))

loss_train = "{:10.4f}".format(mean_squared_error(y_train_sc, y_predictions_train, squared=True))
metric_train = "{:10.4f}".format(mean_absolute_error(y_train_sc, y_predictions_train))

print(metric_test, metric_val, metric_train)

    0.6764     0.6115     0.6913


In [10]:
# without swapping without leakageness - 0.6764     0.6115     0.6913
# with swapping without leakageness -     0.6512     0.6134     0.6229